In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from bs4 import BeautifulSoup
import requests, csv, string, re, datetime
from csv import writer
import time
import random

headers = { 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
cookies = dict(PHPSESSID='hu3bbnbkj9pjhm2h5ls52r5aa0', collections='audio_tech')

In [ ]:
def get_archive(url, base_url, season, offset):
    r  = requests.get(url, headers=headers)
#     print(1)    
    soup = BeautifulSoup(r.text, "html5lib")    
    countries =[]
    
    for l in soup.select('table a'):
        if 'plan.cgi?lang=en&season=' in l.get('href') and '&ci=':
            country = []
            country.append(l.text)
            country.append(l.get('href'))        
            countries.append(country)
        
    listings = []
    countries = countries[offset:]
#     print(countries)
    for c in countries:    
        print(c[0])
        url = base_url + c[1]
        country = c[0]
        r  = requests.get(url, headers=headers)
#         print(2)            
        soup = BeautifulSoup(r.text, "html5lib")
        schedules = soup.find_all('a')
        schedules = [s for s in schedules if 'view.cgi?lang=en&cal=' in s.get('href')]
        for idx,schedule in enumerate(schedules):

            map_dict = {}        
            try:   
                if idx!=0:
                    items = schedule.next_sibling.next_sibling.find_all('a')
                else:
                    items = schedule.parent.next_sibling.find_all('a')
                    
                for item in items:
                    work = item.get_text()
                    composer = re.search('\((.*?)\)', item.parent.next_sibling).group(0)[1:-1]
                    map_dict[work]=composer
                map_dict            
            except:
                None
                
                
            city = schedule.previous_sibling.previous_sibling.find('b').text
            theatre = ','.join(schedule.previous_sibling.previous_sibling.text.split(',')[1:]).strip()        
            url_cal = base_url + schedule.get('href')
            r_cal  = requests.get(url_cal, headers=headers)
#             print(3)

            time.sleep(3)
    
            soup_cal = BeautifulSoup(r_cal.text, "html5lib")
            shows = soup_cal.find_all('b')[8:]
            for show in shows:
                info = []
                info.append(country)
                info.append(city)
                info.append(theatre)                            
                info.append(show.text)
                link = show.parent.parent.parent.find('a').get('href')
                date = link.split("date=")[-1]
                info.append(date)
                
                try:
                    composer = map_dict.get(show.text)
                except:
                    composer = ''
                
                info.append(composer)  
#                 print(map_dict)
#                 print(info)
                listings.append(info)

#     return listings
#     print()
    with open('operabase_archive_' + season + '.csv', 'w') as outfile:
        mywriter = writer(outfile)
        mywriter.writerow(['country', 'city', 'theatre', 'work', 'date', 'composer'])
        for work in listings:
            mywriter.writerow(work) 
        print('completed: ' + season)

In [ ]:
####### URL #########
# get_archive('https://web.archive.org/web/20120510091223/http://operabase.com/plan.cgi?lang=en&season=2011/12', \
#             'https://web.archive.org/web/20120510091223/http://operabase.com/', '1112', 3)
# get_archive('https://web.archive.org/web/20130223084132/http://www.operabase.com/plan.cgi?lang=en&season=2012/13', \
#             'https://web.archive.org/web/20130223084132/http://www.operabase.com/', '1213', 1)
# get_archive('https://web.archive.org/web/20140729224252/http://operabase.com:80/plan.cgi?lang=en&season=2013/14', \
#             'https://web.archive.org/web/20140729224252/http://operabase.com:80/', '1314', 2)
# get_archive('https://web.archive.org/web/20150924162245/http://operabase.com:80/plan.cgi?lang=en&season=2014/15', \
#             'https://web.archive.org/web/20150924162245/http://operabase.com:80/', '1415', 1)
# get_archive('https://web.archive.org/web/20150905064133/http://operabase.com/plan.cgi?lang=en&season=2015/16', \
#             'https://web.archive.org/web/20150905064133/http://operabase.com/', '1516', 1)
# get_archive('https://web.archive.org/web/20160307032948/http://operabase.com/plan.cgi?lang=en&season=2015/16', \
#             'https://web.archive.org/web/20160307032948/http://operabase.com/', '1516', 1)
# get_archive('http://operabase.com/plan.cgi?lang=en&season=2016/17', 'http://operabase.com/', '1617', 1)
# get_archive('http://operabase.com/plan.cgi?lang=en&season=2017/18', 'http://operabase.com/', '1718', 1)
# get_archive('http://operabase.com/plan.cgi?lang=en&season=2018/19', 'http://operabase.com/', '1819', 1)

#maybe?
# 'https://web.archive.org/web/20110703001443/http://operabase.com/plan.cgi?lang=en&season=2010/11&ci='

In [ ]:
items = schedules[0].next_sibling.next_sibling.find_all('a')
map_dict = {}
for item in items:
    work = item.get_text()
    composer = re.search('\((.*?)\)', item.parent.next_sibling).group(0)[1:-1]
    map_dict[work]=composer
#     print(re.search('\((.*?)\)', item.parent.next_sibling).group(0)[1:-1])
map_dict

In [ ]:
from bs4 import BeautifulSoup
import requests, csv, string, re, datetime
from csv import writer

url = 'https://web.archive.org/web/20120524150522/http://operabase.com/plan.cgi?lang=en&season=2011/12&ci=at'
r  = requests.get(url)
soup = BeautifulSoup(r.text, "html5lib")
schedules = soup.find_all('a')
schedules = [s for s in schedules if 'view.cgi?lang=en&cal=' in s.get('href')]
# for schedule in schedules:
#     city = schedule.previous_sibling.previous_sibling.find('b').text
#     theatre = ','.join(schedule.previous_sibling.previous_sibling.text.split(',')[1:]).strip()        
#     url_cal = base_url + schedule.get('href')
# #             print(url_cal)            
#     r_cal  = requests.get(url_cal)
#     soup_cal = BeautifulSoup(r_cal.text, "html5lib")
#     shows = soup_cal.find_all('b')[8:]
#     for show in shows:
#         info = []
#         info.append(country)
#         info.append(city)
#         info.append(theatre)                            
#         info.append(show.text)
#         link = show.parent.parent.parent.find('a').get('href')
#         date = link.split("date=")[-1]
#         info.append(date)
#         listings.append(info)